# Proyecto SQL

# Start up de libros

## Objetivo

Analizaremos una base de datos de una de las aplicaciones para los amantes de los libros. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. 
Esta análisis se utilizará para generar una propuesta para un nuevo producto.


## Preparación de los datos

### Cargar datos

In [1]:
# Cargamos todas las librerías
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Conexión a la base datos
db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# Creamos las consultas para cargar los datos  
consult_books = 'select * from books'
consult_authors = 'select * from authors'
consult_publishers = 'select * from publishers'
consult_ratings = 'select * from ratings'
consult_reviews = 'select * from reviews'

In [4]:
# Cargamos los datasets
books = pd.io.sql.read_sql(consult_books, con = engine)
authors = pd.io.sql.read_sql(consult_authors, con = engine)
publishers = pd.io.sql.read_sql(consult_publishers, con = engine)
ratings = pd.io.sql.read_sql(consult_ratings, con = engine)
reviews = pd.io.sql.read_sql(consult_reviews, con = engine)

### Descripción de los datos

**books**

Contiene datos sobre libros:
* *book_id*: identificación del libro
* *author_id*: identificación del autor o autora
* *title*: título
* *num_pages*: número de páginas
* *publication_date*: fecha de la publicación
* *publisher_id*: identificación de la editorial

**authors**

Contiene datos sobre autores:
* *author_id*: identificación del autor o autora
* *author*: el autor o la autora

**publishers**

Contiene datos sobre editoriales:
* *publisher_id*: identificación de la editorial
* *publisher*: la editorial

**ratings**

Contiene datos sobre las calificaciones de usuarios:
* *rating_id*: identificación de la calificación
* *book_id*: identificación del libro
* *username*: el nombre del usuario que revisó el libro
* *rating*: calificación

**reviews**

Contiene datos sobre las reseñas de los y las clientes:
* *review_id*: identificación de la reseña
* *book_id*: identificación del libro
* *username*: el nombre del usuario que revisó el libro
* *text*: el texto de la reseña


#### books

In [5]:
# Mostramos el DataFrame
display(books.head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


#### authors

In [6]:
# Mostramos el DataFrame
display(authors.head())

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


#### publishers

In [7]:
# Mostramos el DataFrame
display(publishers.head())

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


#### ratings

In [8]:
# Mostramos el DataFrame
display(ratings.head())

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


#### reviews

In [9]:
# Mostramos el DataFrame
display(reviews.head())

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Consulta de datos

### Encuentra el número de libros publicados después del 1 de enero de 2000.


In [10]:
with engine.connect() as connection: 
    query1= ("""
    
        SELECT
            COUNT(books.book_id) as book_count
        FROM
            books
        WHERE
            books.publication_date > '2000-01-01'
            
        """     
    )
    result1 = connection.execute(query1)
    for row in result1:
        print('El número de libros publicados después del 1 de enero de 2000: {}'.format(row['book_count']))

El número de libros publicados después del 1 de enero de 2000: 819


**Observaciones** 

Para 20 años, me parece que son pocos libros publicados, pero es probable que la aplicación sólo tenga los libros más populares del 2000. 

### Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [11]:
# Obtenemos el número de reseñas de usuarios
with engine.connect() as connection: 
    query2= ("""
    
        SELECT
            username,
            COUNT(review_id) as review_count
        FROM
            reviews 
        GROUP BY
            username
        ORDER By
            review_count DESC;
        """     
    )

In [12]:
count_review = pd.io.sql.read_sql(query2, con = engine)
print(count_review)

           username  review_count
0           susan85            29
1       sfitzgerald            28
2        martinadam            27
3       lesliegibbs            26
4            eallen            26
..              ...           ...
155          todd65            10
156        joseph14            10
157        dwayne09            10
158        robert00             9
159  millernicholas             5

[160 rows x 2 columns]


**Observaciones**

El usuario con mayor reseñas es susan85 con 29, y el que hizo menos fue millernicholas con 5. 

In [13]:
# Obtenemos la calificación promedio de cada libro
with engine.connect() as connection: 
    query3= ("""
    
        SELECT
            book_id,
            AVG(rating) as avg_rating
        FROM
            ratings 
        GROUP BY
            book_id
        ORDER BY
            avg_rating DESC;
        """     
    )

In [14]:
avg_rating = pd.io.sql.read_sql(query3, con = engine)
print(avg_rating)

     book_id  avg_rating
0        518        5.00
1        732        5.00
2        347        5.00
3        610        5.00
4        330        5.00
..       ...         ...
995      915        2.25
996      371        2.00
997      316        2.00
998      202        2.00
999      303        1.50

[1000 rows x 2 columns]


**Observaciones** 

* Sólo 1 libro tuvo una calificación de 1.5. 
* Más de la mitad tiene calificación por arriba de 4, y más de 40 libros tienen calificación de 5. 

### Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas

In [15]:
with engine.connect() as connection: 
    query4= ("""
    
        SELECT
            books.publisher_id,
            publishers.publisher,
            COUNT(book_id) as books_published
        FROM
            books
            LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
        WHERE
            num_pages > 50
        GROUP BY
            books.publisher_id,
            publishers.publisher
        ORDER BY
            COUNT(book_id) DESC
        LIMIT
            1;
        """     
    )
    result4 = connection.execute(query4)
    for row in result4:
        print('La editorial que ha publicado más libros de 50 páginas es {} con {} libros publicados'.format(row['publisher'],row['books_published']))

La editorial que ha publicado más libros de 50 páginas es Penguin Books con 42 libros publicados


**Observaciones** 

Tiene lógica que Penguin Books sea la editorial resultante, siendo que es la editorial líder en publicar literatura clásica por más de 75 años. 

### Identifica al autor que tiene la más alta calificación promedio del libro


In [16]:
# Mira solo los libros con al menos 50 calificaciones.
with engine.connect() as connection: 
    query5= ("""
    
        SELECT
            books.author_id AS author_id,
            AVG(ratings.rating) AS average_rating,
            COUNT(ratings.book_id) AS rated_books,
            authors.author AS author
        FROM
            books
            INNER JOIN ratings ON ratings.book_id = books.book_id
            INNER JOIN authors ON authors.author_id = books.author_id
        GROUP BY
            books.author_id,
            authors.author
        HAVING 
            COUNT(ratings.book_id) >= 50
        ORDER BY
            average_rating DESC
        LIMIT
            1
        """     
    )
    result5 = connection.execute(query5)
    for row in result5:
        print('El autor con más alta calificación promedio es {}'.format(row['author']))


El autor con más alta calificación promedio es Diana Gabaldon


**Observaciones**

Con libros como Outlander que se han hecho series de televisión, Diana Gabaldon es la autora cuyo(s) libro(s) tienen la calificación promedio más alta.

### Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [19]:
# Sacamos el promedio
with engine.connect() as connection: 
    query8= ("""
    
        SELECT
            AVG(sub2.review_count) AS avg_review_count
        FROM(
            SELECT
                COUNT(reviews.review_id) AS review_count,
                sub1.username
            FROM
                (
                SELECT 
                    ratings.username, 
                    COUNT(rating_id) AS count_rating 
                FROM 
                    ratings
                GROUP BY 
                    ratings.username
                HAVING 
                    COUNT(ratings.rating_id) >50
                )AS sub1
            INNER JOIN reviews ON reviews.username = sub1.username
            GROUP BY
                sub1.username

        )AS sub2;       
        """
    )
    result8 = connection.execute(query8)
    for row in result8:
        print('El número promedio de reseñas de texto es {}'.format(row['avg_review_count']))

El número promedio de reseñas de texto es 24.3333333333333333


**Observaciones**

Los usuarios son bastante activos, dedicaron tiempo para dejar un promedio de 211 reseñas.

## Conclusiones

* Podemos en enfocarnos en libros contemporaneos y que tengan altas calificaciones para atraer a los usuarios
* Penguin Books sería una de las principales editoriales con las cuales habría que contar en nuestro servicio. 
* De igual forma autoras como Diana Gabaldon.
* Podemos implementar algun tipo de recompensa para los usuarios por dejar una reseña, de esta forma podríamos atraer a más usuarios.  